In [ ]:
import numpy as np
import pandas as pd

In [ ]:
%%bash
head -1 /mnt/idms/bdavid/projects/twitter-comb-occ-2017/rg17/rg17_8000_important_en_words_plus_players_results.txt

# Load raw combined occurences

In [ ]:
#file_path = "/mnt/idms/bdavid/projects/twitter-comb-occ-2017/rg17/rg17_combOcc_results_20170922.txt"
#file_path = "/mnt/idms/bdavid/projects/twitter-comb-occ-2017/rg17/rg17_8000_important_en_words_plus_players_results.txt"
file_path = "/mnt/idms/bdavid/projects/twitter-comb-occ-2017/rg17/rg17_8000_en-players_6h_accurate.txt"
#occurences_raw_pd = pd.read_csv(file_path, sep=",", header=None)#, nrows=1000)

In [ ]:
num_cols = 203
raw_rows = []
with open(file_path) as f:
    i = 0
    for line in f:
        i += 1
        splitted = line.rstrip().split(",")
        if len(splitted) == 4:
            # no combined occurences
            continue
        row = list(np.nan for i in range(num_cols))
        row[:len(splitted)] = splitted
        raw_rows.append(tuple(row))
        #if i > 10:
        #    break

In [ ]:
occurences_raw_pd = pd.DataFrame(raw_rows)

In [ ]:
len(occurences_raw_pd)

In [ ]:
occurences_raw_pd.head(5)

# Clean columns

In [ ]:
def clean_cell(x):
    out = x
    try:
        if not x is np.nan:
            for pattern in ["Some(","List(","(",")"," "]:
                if pattern in out:
                    out = out.replace(pattern,"")
    except:
        print(x)
        raise
    return out

In [ ]:
occurences_pd = occurences_raw_pd.applymap(clean_cell)
del occurences_pd[0]

In [ ]:
num_key_words = 100
kw_text_cols = [(i+1)*2+1 for i in range(num_key_words)]
kw_count_cols = [(i+1)*2+2 for i in range(num_key_words)]

In [ ]:
occurences_pd[kw_count_cols] = occurences_pd[kw_count_cols].astype("float64")

In [ ]:
occurences_pd.head(5)

In [ ]:
occurences_pd.to_csv("/mnt/idms/fberes/network/roland_garros/combined_occ_parsed.csv",sep="|",index=False, header=False)